Librerias

In [1]:
import pandas as pd
import numpy as np

Lectura de los datos de entrada.

In [3]:
# Datos de entrada
Filename_Exp = 'Input/Expansion_con_alimentadores.xlsx'  # Datos expansión territorial.
Filename_Demanda = {
    665: 'Input/MEDIDAS_665.xlsx',
    675: 'Input/MEDIDAS_675.xlsx',
    677: 'Input/MEDIDAS_677.xlsx',
    678: 'Input/MEDIDAS_678.xlsx'
}

# Cargar datos de expansión territorial y agregar columna de alimentador si es necesario.
columnas_interes_exp = ['Longitud', 'Latitud', 'tot(2017)', 'Alimentador Cercano','Bus Cercano','Nombre Bus Cercano']
personas_df = pd.read_excel(Filename_Exp, usecols=columnas_interes_exp)

# Inicializar DataFrame para almacenar los resultados finales.
resultados_final = pd.DataFrame()

# Procesar los datos para cada alimentador
for alimentador, filename in Filename_Demanda.items():
    # Leer el archivo de medidas correspondiente al alimentador actual.
    medidas_df = pd.read_excel(filename)
    
    # Filtrar personas_df para el alimentador actual.
    personas_alimentador = personas_df[personas_df['Alimentador Cercano'] == alimentador]
    
    # Sumar el total de viviendas para el alimentador actual.
    total_viviendas_alimentador = personas_alimentador['tot(2017)'].sum()
    
    # Crear un DataFrame para almacenar los resultados de este alimentador.
    resultados_alimentador = []
    
    # Iterar sobre cada registro de medidas para el alimentador actual.
    for _, row_medidas in medidas_df.iterrows():
        fecha_local = row_medidas['fecha_local']
        p = row_medidas['p']
        q = row_medidas['q']
        s = row_medidas['s']
        
        # Calcular la potencia base para el alimentador actual.
        potencia_base_total_p = p / total_viviendas_alimentador
        potencia_base_total_q = q / total_viviendas_alimentador
        potencia_base_total_s = s / total_viviendas_alimentador
        
        # Distribuir la potencia a cada vivienda para el alimentador actual.
        for _, row_personas in personas_alimentador.iterrows():
            longitud = row_personas['Longitud']
            latitud = row_personas['Latitud']
            num_total = row_personas['tot(2017)']
            bus_cercano = row_personas['Bus Cercano']
            nombre_bus_cercano = row_personas['Nombre Bus Cercano']
            potencia_asignada_total_p = potencia_base_total_p * num_total
            potencia_asignada_total_q = potencia_base_total_q * num_total
            potencia_asignada_total_s = potencia_base_total_s * num_total
            
            # Agregar los resultados a la lista de resultados del alimentador.
            resultados_alimentador.append([
                longitud, latitud, num_total, potencia_asignada_total_p, potencia_asignada_total_q, 
                potencia_asignada_total_s, fecha_local, alimentador, bus_cercano,nombre_bus_cercano
            ])
    
    # Convertir los resultados del alimentador a DataFrame y agregarlos al DataFrame final.
    columnas_resultados = ['Longitud', 'Latitud', 'tot(2017)', 'p', 'q', 's', 'fecha_local', 'Alimentador','Bus Cercano', 'Nombre Bus Cerano']
    resultados_alimentador_df = pd.DataFrame(resultados_alimentador, columns=columnas_resultados)
    resultados_final = pd.concat([resultados_final, resultados_alimentador_df])

# Guardar el DataFrame final en un archivo Excel.
resultados_final.to_excel('Output/Consumo_por_alimentador.xlsx', index=False)

4592
86443
32337
6317
